## 전처리

In [1]:
import pandas as pd
import os

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
print('데이터 크기 :',orginal_data_size)
ratings.head()

데이터 크기 : 1000209


,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
all = pd.read_csv(rating_file_path, sep='::',  engine='python', encoding = "ISO-8859-1")
all.head()
#원본모양확인, 열의 이름도 없지만 위에서 표기해줬다

,1,1193,5,978300760
0,1,661,3,978302109
1,1,914,3,978301968
2,1,3408,4,978300275
3,1,2355,5,978824291
4,1,1197,3,978302268


In [3]:
# 3점 이상만 남깁니다. 3점이상만 선호한다고 가정
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [5]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


## 2) 분석해 봅시다.

    ratings에 있는 유니크한 영화 개수
    ratings에 있는 유니크한 사용자 수
    가장 인기 있는 영화 30개(인기순)

In [7]:
# 무비에 적용해보기
from scipy.sparse import csr_matrix

num_movie = ratings['movie_id'].nunique()
num_user = ratings['user_id'].nunique()

print(f'ratings에 있는 유니크한 영화 개수:{num_movie} 개 \nratings에 있는 유니크한 사용자 수:{num_user} 명')

ratings에 있는 유니크한 영화 개수:3628 개 
ratings에 있는 유니크한 사용자 수:6039 명


In [8]:
use_cols = ['user_id','movie_id','counts']#쓸 열만 남겨둔다
ratings = ratings[use_cols]

In [9]:
# 인기 많은 영화 아직 id다 id를 영화 명으로 변환해줘야겠는데...
movie_count = ratings.groupby('movie_id')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)


movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

In [10]:
mov_top= movie_count.sort_values(ascending=False).head(30)
mov_top.values #무비아이디 하나당 유저수

array([3211, 2910, 2885, 2716, 2561, 2509, 2498, 2473, 2460, 2434, 2413,
       2385, 2371, 2314, 2297, 2257, 2252, 2213, 2210, 2194, 2167, 2121,
       2102, 2066, 2051, 2030, 2022, 2019, 2000, 1941])

In [11]:
top = mov_top.index #이게 영화아이디
top

Int64Index([2858,  260, 1196, 1210, 2028,  589,  593, 1198, 1270, 2571,  480,
            2762,  608,  110, 1580,  527, 1197, 2396, 1617,  318,  858, 1265,
            1097, 2997, 2716,  296,  356, 1240,    1,  457],
           dtype='int64', name='movie_id')

In [12]:
len(top)

30

In [13]:
mc = movies[movies['movie_id'].isin([top[0]])]
for i in range(1,30):
    mc = pd.concat([mc,movies[movies['movie_id'].isin([top[i]])]], ignore_index=True) #인덱스 무시하고 하나씩 밑에 붙여줬다
    #이렇게 번거롭게한 이유는 그냥 isin으로 검색할 경우 기껏 영화 카운트 수대로 정렬한게 없어지고 movie_id 순으로 나와버린다
mc

,movie_id,title,genre
0,2858,American Beauty (1999),Comedy|Drama
1,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
2,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
3,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
4,2028,Saving Private Ryan (1998),Action|Drama|War
5,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
6,593,"Silence of the Lambs, The (1991)",Drama|Thriller
7,1198,Raiders of the Lost Ark (1981),Action|Adventure
8,1270,Back to the Future (1985),Comedy|Sci-Fi
9,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller


In [14]:
ct = pd.Series(mov_top.values, name = 'count')
#count 항목 만들기


In [15]:
md = pd.concat([mc,ct],axis=1) # 우측에 새로 붙이고
md
#와.. 이 과정이 이거안되고 저거 안되고 ㅠ 엄청 오래걸렸습니다 다음엔 데이터 내용을 맘대로 뽑아 맘대로 더할수 있겠죠?
# 가장 인기 영화 30개 목록

,movie_id,title,genre,count
0,2858,American Beauty (1999),Comedy|Drama,3211
1,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,2910
2,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War,2885
3,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War,2716
4,2028,Saving Private Ryan (1998),Action|Drama|War,2561
5,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller,2509
6,593,"Silence of the Lambs, The (1991)",Drama|Thriller,2498
7,1198,Raiders of the Lost Ark (1981),Action|Adventure,2473
8,1270,Back to the Future (1985),Comedy|Sci-Fi,2460
9,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,2434


## 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.
위에서 보니 본 영화가 좀 보인다 589 1198 2571 480 1580

In [16]:
my_mov_name = [movies.loc[589]['title'],movies.loc[1198]['title'],movies.loc[2571]['title'],movies.loc[480]['title'],movies.loc[1580]['title']]
my_mov_id = [movies.loc[589]['movie_id'],movies.loc[1198]['movie_id'],movies.loc[2571]['movie_id'],movies.loc[480]['movie_id'],movies.loc[1580]['movie_id']]
print(my_mov_name)
#노가다로 만들고보니 리스트 만들어서 넣을껄 했다
list = [589, 1198, 2571, 480, 1580]
nm = movies.loc[list]['title']
print(nm.values) #리스트로도 만듬 ㅠ 어 왜 내가 고른거랑 틀리지

['Silence of the Lambs, The (1991)', 'Big Blue, The (Le Grand Bleu) (1988)', 'Superman (1978)', 'Lassie (1994)', 'Wishmaster (1997)']
['Silence of the Lambs, The (1991)' 'Big Blue, The (Le Grand Bleu) (1988)'
 'Superman (1978)' 'Lassie (1994)' 'Wishmaster (1997)']


In [17]:
movies.loc[589] #무비 id번호랑 이름 주소가 틀리다 무비 아이디로 찾아야되는데

movie_id                                 593
title       Silence of the Lambs, The (1991)
genre                         Drama|Thriller
Name: 589, dtype: object

In [18]:
movies[movies.movie_id == 589]

,movie_id,title,genre
585,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller


In [19]:
movies[movies['movie_id'].isin(list)] #이제 내가 고른 영화들의 인덱스 값이 보인다


,movie_id,title,genre
476,480,Jurassic Park (1993),Action|Adventure|Sci-Fi
585,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
1180,1198,Raiders of the Lost Ark (1981),Action|Adventure
1539,1580,Men in Black (1997),Action|Adventure|Comedy|Sci-Fi
2502,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller


In [20]:
list2 = [476,585,1180,1539,2502] #인덱스 번호
nm = movies.loc[list2]['title']
my_mov_name = nm.values
my_mov_id = nm.index
print(my_mov_name, my_mov_id)

['Jurassic Park (1993)' 'Terminator 2: Judgment Day (1991)'
 'Raiders of the Lost Ark (1981)' 'Men in Black (1997)'
 'Matrix, The (1999)'] Int64Index([476, 585, 1180, 1539, 2502], dtype='int64')


In [21]:
ratings.tail(10)# 마지막 id는 6040

,user_id,movie_id,counts
1000198,6040,2021,3
1000199,6040,2022,5
1000200,6040,2028,5
1000201,6040,1080,4
1000202,6040,1089,4
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4


In [22]:
my_mov_list= pd.DataFrame({'user_id' : [6041]*5, 'movie_id' : list, 'counts' : [5,5,4,4,5]}) #데이터 프레임 짜서
if not ratings['user_id'].isin([6041]).any():  # user_id에 '6041'이라는 데이터가 없다면
    ratings = ratings.append(my_mov_list) #레이팅의 밑에 더한다
print(ratings.tail(10))

         user_id  movie_id  counts
1000203     6040      1090       3
1000205     6040      1094       5
1000206     6040       562       5
1000207     6040      1096       4
1000208     6040      1097       4
0           6041       589       5
1           6041      1198       5
2           6041      2571       4
3           6041       480       4
4           6041      1580       5


In [23]:
#ratings.reset_index(inplace=True) #인덱스 리셋해볼랬더니 인덱스는 안건들고 새로운 인덱스가 생겼다 봉인
#ratings.tail(10)

## 4) CSR matrix를 직접 만들어 봅시다.

In [24]:
print(num_user,num_movie) #rating에서 뽑아놓은 넘버

num_momovie = movies['movie_id'].nunique() #movie에서의 유니크한 영화수
print(num_momovie, num_movie) # 3883, 3628로 좀 차이가 난다

6039 3628
3883 3628


In [25]:
for c in ratings.columns: #결측치 수 확인
    print('{} : {}'.format(c, len(ratings.loc[pd.isnull(ratings[c]), c].values)))

user_id : 0
movie_id : 0
counts : 0


In [26]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

# 유저, 영화indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_userdata = ratings['user_id'].map(user_to_idx.get).dropna(axis=0)
if len(temp_userdata) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id_temp'] = temp_userdata   # ratings['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_moviedata = ratings['movie_id'].map(movie_to_idx.get).dropna(axis=0)
if len(temp_moviedata) == len(ratings):
    print('movie_id column indexing OK!!')
    ratings['movie_id_temp'] = temp_moviedata
else:
    print('movie_id column indexing Fail!!')


user_id column indexing OK!!
movie_id column indexing OK!!


In [27]:
print(num_user,num_movie) #rating에서 뽑아놓은 넘버
print(max(ratings['user_id_temp']),max(ratings['movie_id_temp'])) #csr_matrix에서는 이런식으로 맥스값을 받아와서해서 실제 넘버랑 요구치가 다르다 +1해야 안터진다

6039 3628
6039 3627


In [28]:
csr_data = csr_matrix((ratings['counts'], (ratings['user_id_temp'], ratings['movie_id_temp'])), 
                      shape=(max(ratings['user_id_temp'])+1, max(ratings['movie_id_temp'])+1))
print(csr_data)
#로우 인덱스가 자꾸 터진다
#어디서 터지는지 못찾겠다 결측치도 찾고 혹시 몰라서 카운트쪽도 해봤는데...
#csr_matrix에서 max인덱스 값을 받는바람에 맥스값 받아서 1씩올렸다

  (0, 0)	5
  (0, 1)	3
  (0, 2)	3
  (0, 3)	4
  (0, 4)	5
  (0, 5)	3
  (0, 6)	5
  (0, 7)	5
  (0, 8)	4
  (0, 9)	4
  (0, 10)	5
  (0, 11)	4
  (0, 12)	4
  (0, 13)	4
  (0, 14)	5
  (0, 15)	4
  (0, 16)	3
  (0, 17)	4
  (0, 18)	5
  (0, 19)	4
  (0, 20)	3
  (0, 21)	3
  (0, 22)	5
  (0, 23)	5
  (0, 24)	3
  :	:
  (6038, 2311)	4
  (6038, 2317)	5
  (6038, 2386)	4
  (6038, 2394)	5
  (6038, 2424)	4
  (6038, 2437)	4
  (6038, 2446)	5
  (6038, 2471)	4
  (6038, 2511)	5
  (6038, 2523)	4
  (6038, 2559)	3
  (6038, 2560)	4
  (6038, 2631)	5
  (6038, 2648)	4
  (6038, 2654)	5
  (6038, 2738)	4
  (6038, 2740)	5
  (6038, 2857)	5
  (6038, 2860)	3
  (6038, 3311)	5
  (6039, 92)	5
  (6039, 107)	4
  (6039, 120)	5
  (6039, 124)	4
  (6039, 175)	5


## 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [29]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=200, regularization=0.01, 
                                    use_gpu=False, iterations=20, dtype=np.float32)

# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/20 [00:00<?, ?it/s]

## 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.
### 영화고를경우 관련 추천

In [30]:
new_mov = "Terminator 2: Judgment Day (1991)"
#new_mov = "Back to the Future (1985)" #바꿔봄
new_mov_id = movies[movies['title'] == new_mov]
print(new_mov_id)

     movie_id                              title                   genre
585       589  Terminator 2: Judgment Day (1991)  Action|Sci-Fi|Thriller


In [31]:
print(new_mov_id.values.tolist()[0])
new_mov_id = new_mov_id.values.tolist()[0][0]
new_mov_id_ratings = movie_to_idx[new_mov_id]
similar_movies = als_model.similar_items(new_mov_id_ratings, N=10)
similar_movies #10개 추천받기

[589, 'Terminator 2: Judgment Day (1991)', 'Action|Sci-Fi|Thriller']


[(92, 0.99999994),
 (124, 0.64097196),
 (200, 0.59431845),
 (62, 0.54232466),
 (107, 0.461116),
 (175, 0.42824772),
 (141, 0.41718158),
 (2739, 0.34980878),
 (3302, 0.3384693),
 (3280, 0.3322994)]

In [32]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()} #인덱스로부터 movie_id 값을 다시 받아온다
recommand_list = [idx_to_movie[i[0]] for i in similar_movies]
rec_score = [i[1] for i in similar_movies]
print(recommand_list) #추천리스트
print(rec_score) #스코어 값도 따로 남겨둔다

[589, 2571, 1240, 2916, 480, 1580, 457, 3562, 245, 1369]
[0.99999994, 0.64097196, 0.59431845, 0.54232466, 0.461116, 0.42824772, 0.41718158, 0.34980878, 0.3384693, 0.3322994]


In [33]:
similar_movies[0][1]

0.99999994

In [34]:
final = movies[movies.movie_id == recommand_list[0]] #위에서 했던 만들어 두고 뒤에 붙여넣기
for i in range(1,len(recommand_list)):
    final = pd.concat([final,movies[movies.movie_id == recommand_list[i]]],ignore_index=True)
score = pd.Series(rec_score, name = 'score')#스코어 열 생성
final = pd.concat([final,score],axis=1)# 붙이기
print(final)
#0.64이 두번쨰라.. 괜찮네요 제 선호 영화에도 넣어둔 매트릭스가 2위

   movie_id                                    title  \
0       589        Terminator 2: Judgment Day (1991)   
1      2571                       Matrix, The (1999)   
2      1240                   Terminator, The (1984)   
3      2916                      Total Recall (1990)   
4       480                     Jurassic Park (1993)   
5      1580                      Men in Black (1997)   
6       457                     Fugitive, The (1993)   
7      3562                         Committed (2000)   
8       245                 Glass Shield, The (1994)   
9      1369  I Can't Sleep (J'ai pas sommeil) (1994)   

                              genre     score  
0            Action|Sci-Fi|Thriller  1.000000  
1            Action|Sci-Fi|Thriller  0.640972  
2            Action|Sci-Fi|Thriller  0.594318  
3  Action|Adventure|Sci-Fi|Thriller  0.542325  
4           Action|Adventure|Sci-Fi  0.461116  
5    Action|Adventure|Comedy|Sci-Fi  0.428248  
6                   Action|Thriller  0.417182  

In [81]:
def get_similar_movies(movie_name: str):
    movie_id = movies[movies['title'] == movie_name]
    movie_id = movie_id.values.tolist()[0][0]
    nmovie_id = movie_to_idx[movie_id]
    similar_movie = als_model.similar_items(nmovie_id)
    
    return similar_movie

def view(similar_movie):
    idx_to_movie = {v:k for k,v in movie_to_idx.items()} #인덱스로부터 movie_id 값을 다시 받아온다
    similar_movie = [idx_to_movie[i[0]]for i in similar_movie]
    final = movies[movies.movie_id == similar_movie[0]] #위에서 했던 만들어 두고 뒤에 붙여넣기
    for i in range(1,len(similar_movie)):
        final = pd.concat([final,movies[movies.movie_id == similar_movie[i]]],ignore_index=True)
    return final
#함수 자동화 하려고 했는데 IndexError: invalid index to scalar variable 에러때문에 스코어를 못넣었습니다
#스택오버플로우 조언보고 고쳐도 답이 없네요 ㅠ

In [82]:
get_similar_movies(f'Terminator 2: Judgment Day (1991)')

[(92, 0.99999994),
 (124, 0.64097196),
 (200, 0.59431845),
 (62, 0.54232466),
 (107, 0.461116),
 (175, 0.42824772),
 (141, 0.41718158),
 (2739, 0.34980878),
 (3302, 0.3384693),
 (3280, 0.3322994)]

In [83]:
get_similar_movies(f'Jurassic Park (1993)')

[(107, 0.9999999),
 (175, 0.5596884),
 (92, 0.461116),
 (82, 0.36706743),
 (62, 0.3448405),
 (1713, 0.30355424),
 (3151, 0.27519128),
 (1181, 0.26962206),
 (2129, 0.26914603),
 (2748, 0.268756)]

In [84]:
view(get_similar_movies(f'Terminator 2: Judgment Day (1991)'))

,movie_id,title,genre
0,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
1,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller
2,1240,"Terminator, The (1984)",Action|Sci-Fi|Thriller
3,2916,Total Recall (1990),Action|Adventure|Sci-Fi|Thriller
4,480,Jurassic Park (1993),Action|Adventure|Sci-Fi
5,1580,Men in Black (1997),Action|Adventure|Comedy|Sci-Fi
6,457,"Fugitive, The (1993)",Action|Thriller
7,3562,Committed (2000),Comedy|Drama
8,245,"Glass Shield, The (1994)",Crime|Drama
9,1369,I Can't Sleep (J'ai pas sommeil) (1994),Drama|Thriller


In [39]:
view(get_similar_movies(f'Jurassic Park (1993)'))

,movie_id,title,genre
0,480,Jurassic Park (1993),Action|Adventure|Sci-Fi
1,1580,Men in Black (1997),Action|Adventure|Comedy|Sci-Fi
2,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
3,1544,"Lost World: Jurassic Park, The (1997)",Action|Adventure|Sci-Fi|Thriller
4,2916,Total Recall (1990),Action|Adventure|Sci-Fi|Thriller
5,893,Mother Night (1996),Drama
6,3845,And God Created Woman (Et Dieu&#8230;Créa la F...,Drama
7,804,She's the One (1996),Comedy|Romance
8,1860,Character (Karakter) (1997),Drama
9,811,"Bewegte Mann, Der (1994)",Comedy


### 유저에게 아티스트 추천하기
AlternatingLeastSquares 클래스에 구현되어 있는 recommend 메서드를 통하여 제가 좋아할 만한 아티스트를 추천받습니다. filter_already_liked_items 는 유저가 이미 평가한 아이템은 제외하는 Argument입니다.

In [40]:
ratings.tail(10) #csr을 위한 과정에서 유저id와 movie_id가 변형된다

,user_id,movie_id,counts,user_id_temp,movie_id_temp
1000203,6040,1090,3,6038,1030
1000205,6040,1094,5,6038,986
1000206,6040,562,5,6038,311
1000207,6040,1096,4,6038,142
1000208,6040,1097,4,6038,26
0,6041,589,5,6039,92
1,6041,1198,5,6039,120
2,6041,2571,4,6039,124
3,6041,480,4,6039,107
4,6041,1580,5,6039,175


In [41]:
user = user_to_idx[6041] #접니다
print(user)
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=10, filter_already_liked_items=True) #이미 선호 고른 아이템은 제외
movie_recommended

6039


[(62, 0.41039938),
 (44, 0.39339766),
 (172, 0.34009454),
 (200, 0.3070259),
 (117, 0.2971191),
 (141, 0.27311572),
 (145, 0.2489782),
 (150, 0.2166046),
 (375, 0.21131411),
 (87, 0.20336099)]

In [43]:
[idx_to_movie[i[0]] for i in movie_recommended] 
#본문건데 내 함수(view)로 대체했다

[2916, 260, 1291, 1240, 1196, 457, 1527, 780, 1573, 110]

In [42]:
view(movie_recommended) #유저 6041(나)에 추천된 영화 리스트
#꾸미는 것이 아니라 내취향에 잘맞는다 sci-fi, 액션, 어드벤쳐 대부분 본것들이고 좋아한다

,movie_id,title,genre
0,2916,Total Recall (1990),Action|Adventure|Sci-Fi|Thriller
1,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
2,1291,Indiana Jones and the Last Crusade (1989),Action|Adventure
3,1240,"Terminator, The (1984)",Action|Sci-Fi|Thriller
4,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
5,457,"Fugitive, The (1993)",Action|Thriller
6,1527,"Fifth Element, The (1997)",Action|Sci-Fi
7,780,Independence Day (ID4) (1996),Action|Sci-Fi|War
8,1573,Face/Off (1997),Action|Sci-Fi|Thriller
9,110,Braveheart (1995),Action|Drama|War


In [49]:
movies[movies['movie_id'].isin(list)] #내가 고른 선호영화들


,movie_id,title,genre
476,480,Jurassic Park (1993),Action|Adventure|Sci-Fi
585,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
1180,1198,Raiders of the Lost Ark (1981),Action|Adventure
1539,1580,Men in Black (1997),Action|Adventure|Comedy|Sci-Fi
2502,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller


In [53]:
movies[movies.movie_id == 2916] #토탈리콜을 1위로 추천해주고있다

,movie_id,title,genre
2847,2916,Total Recall (1990),Action|Adventure|Sci-Fi|Thriller


In [67]:
ratings[ratings.movie_id == 2916] #2916이 62로 변형되었다

,user_id,movie_id,counts,user_id_temp,movie_id_temp
62,2,2916,3,1,62
525,7,2916,5,6,62
562,8,2916,5,7,62
1415,13,2916,3,12,62
1597,15,2916,4,14,62
...,...,...,...,...,...
997482,6024,2916,4,6022,62
997844,6026,2916,4,6024,62
997948,6028,2916,3,6026,62
998059,6030,2916,4,6028,62


In [68]:
explain = als_model.explain(user, csr_data, itemid=62) #설명

In [87]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]] #영화 인덱스로 변환, 터미네이터와 맨인블랙,매트릭스,주라기공원의 영향을 받아서 추천되었다
#장르로 볼때 액션,싸이파이,어드벤쳐,스릴러에 맞게 된듯하다. 공통점으로 액션과 싸이파이가 두드러진다

[(589, 0.19055988554044156),
 (1580, 0.11163803887183592),
 (2571, 0.08665609327386052),
 (480, 0.06008652701004547),
 (1198, -0.04551208518564635)]

In [70]:
view([(i[0], i[1]) for i in explain[1]]) #시각화

,movie_id,title,genre
0,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
1,1580,Men in Black (1997),Action|Adventure|Comedy|Sci-Fi
2,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller
3,480,Jurassic Park (1993),Action|Adventure|Sci-Fi
4,1198,Raiders of the Lost Ark (1981),Action|Adventure


|No.|평가문항|상세기준|
|-|:-|:-|
|1.| CSR matrix가 정상적으로 만들어졌다.|	사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.|
|2.| MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.|	사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.|
|3.| 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.|	MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.|

# 회고
중간중간에 막힌 부분과 감상은 위에 다 적었습니다.    
자주쓰는 판다스나 시리즈 같은 형태 변환 풀고 압축하는 그런게 아직도 어렵네요    
시간 한참 더들였지만 결국 인덱스 스칼라 에러 못잡아서 스코어 못붙인게 아쉽네요